# qsiparc volume parcellation demo

This notebook exercises the current volume-only parcellation stub with a user-supplied atlas path and a synthetic scalar map.
Right now the strategy just counts scalar maps per subject; it is a scaffold for real voxel-to-label aggregation.

In [ ]:
from pathlib import Path

from qsiparc.atlas.registry import AtlasResource
from qsiparc.io.data_models import AtlasDefinition, ReconInput, SubjectContext
from qsiparc.parcellation.pipeline import ParcellationPlan, VolumeParcellator
from qsiparc.parcellation.strategies import VolumeParcellationStrategy

BASE = Path("demo_data")
BASE.mkdir(exist_ok=True)

# Pretend atlas + scalar map files
atlas_path = BASE / "demo_atlas.nii.gz"
scalar_path = BASE / "sub-01_demo_fa.nii.gz"
atlas_path.touch()
scalar_path.touch()

atlas_def = AtlasDefinition(name="demo-atlas", path=atlas_path, labels={})
atlas_res = AtlasResource(definition=atlas_def)

recon_input = ReconInput(
    context=SubjectContext(subject_id="01"),
    scalar_maps={"FA": scalar_path},
    mask=None,
    transforms=(),
)

strategy = VolumeParcellationStrategy()
parcellator = VolumeParcellator(strategy=strategy)
plan = ParcellationPlan(atlas=atlas_res, inputs=[recon_input], strategy=strategy)
result = parcellator.run(plan)

result

A minimal example of a metric consuming the `ParcellationResult`. Replace this with real computations as metrics come online.

In [ ]:
from dataclasses import dataclass

from qsiparc.metrics.summary import RegionMetric


@dataclass(frozen=True)
class DummyRegionMetric(RegionMetric):
    name: str = "dummy"

    def compute(self, result):
        return len(result.region_summaries)


metric = DummyRegionMetric()
metric.compute(result)